In [1]:
from pyalex import (
    Works, Authors, Sources,
    Institutions, Concepts, Publishers, Funders
)
import pyalex
import pandas as pd
import numpy as np
pyalex.config.email = "david@rs21.io"

from flair.embeddings import DocumentPoolEmbeddings
from flair.data import Sentence
from flair.embeddings import SentenceTransformerDocumentEmbeddings

EMBEDDING_MODEL_1 = "all-mpnet-base-v2" 

# this one is also good: all-MiniLM-L6-v2
EMBEDDING_MODEL_2 = "all-MiniLM-L6-v2"
SENT_EMBEDDINGS_1 = SentenceTransformerDocumentEmbeddings(EMBEDDING_MODEL_1)
SENT_EMBEDDINGS_2 = SentenceTransformerDocumentEmbeddings(EMBEDDING_MODEL_2)
DOC_EMBEDDINGS= DocumentPoolEmbeddings([SENT_EMBEDDINGS_2])

import torch
from tqdm import tqdm
import yake
import umap.umap_ as umap
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture as GMM
import altair as alt
import math
import plotly.express as px
import textwrap

In [2]:
search_term = ('data link' + 
              '|near real time')
datalink_concepts = Concepts().\
search_filter(display_name=search_term).get()

In [3]:
concepts = []
for i in range(len(datalink_concepts)):
    id_, display_name = datalink_concepts[i]['id'], datalink_concepts[i]['display_name']
    concepts.append((id_, display_name))
concepts

[('https://openalex.org/C69075417', 'Linked data'),
 ('https://openalex.org/C192835768', 'Data link layer'),
 ('https://openalex.org/C182132293', 'Data link')]

In [4]:
institution = "Beijing University of Posts and Telecommunications"

In [5]:
def process_works_list(worklist:list):
    """
    transforms the 
    works list into a dataframe.
    """
    abstracts_dict = {h["id"]:h["abstract"] for h in worklist}
    df = pd.DataFrame.from_records(worklist)
    try: 
        del df['abstract_inverted_index'] # though don't all have abstracts is the problem
        df['abstract'] = df['id'].map(abstracts_dict)
    except:
        pass
   # df['author_affils'] = df['authorships'].apply(get_authors_and_affils)
    return df

https://github.com/J535D165/pyalex

In [6]:
from pyalex import config

config.max_retries = 0
config.retry_backoff_factor = 0.1
config.retry_http_codes = [429, 500, 503]

In [7]:
bupt = Institutions().search_filter(display_name= "Beijing University of Posts and Telecommunications").get()

In [8]:
type(bupt)

list

In [21]:
# Works().filter(authorships={"institutions": {"ror": "04w9fbh59"}}).get()

In [9]:
#bupt[0]

In [10]:
#Works().search_filter(institutions=bupt).get()

In [12]:
#Works().filter(authorships={"institutions": {"ror": "04w9fbh59"}}).get()

In [17]:
def get_ror_frame(ror:str):
    """
    takes a list of Concepts() results and an index
    and forms the pagination object to retrive the 
    records
    """
    pager = Works().filter(publication_year='>2018',
    authorships={"institutions": {"ror": ror}}).paginate(
        per_page=200, n_max=None)
    #pager = Works().filter(publication_year='>2016',
    #concepts={"id":f"{concepts_list[i]['id']}"}).filter(authorships={"institutions":{"country_code":"CN"}}).\
    #paginate(per_page=200,n_max=None)
    #concepts={"id":f"{concepts_list[i]['id']}"}).\
    #paginate(per_page=200,n_max=None)
    df = pd.DataFrame()
    for page in tqdm(pager):
        dfpage = process_works_list(page)
        df = pd.concat([df, dfpage], ignore_index=True)
        df.drop_duplicates(subset='id', keep='first',inplace=True)
    return df

In [18]:
df = get_ror_frame("04w9fbh59")

124it [03:50,  1.86s/it]


In [19]:
df.shape

(24425, 49)

In [21]:
#dftop = pd.concat(df,
#                  ignore_index=True)
dftop = df
dftop.drop_duplicates(subset='id', keep='first', 
                      inplace=True)

dftop.set_index('id', inplace=True, drop=False)

dfall = dftop
print(dfall.shape)

dfall['content'] = dfall['title'] + ". " + dfall['abstract']

dfrecords = dfall[~dfall['content'].isna()].copy()

(24425, 49)


In [22]:
def get_keywords(text:str, top:int=7, stopwords=None):
    """
    takes a blob of text and 
    returns the top **top** 
    keywords as a list
    """
    kw_extractor = yake.KeywordExtractor(top=top, stopwords=stopwords)
    keywords = kw_extractor.extract_keywords(text)
    return [p[0] for p in keywords]

In [23]:
def get_top_concepts(concept_list:list,score:float=.6):
    """
    takes a list of concept dictionaries 
    returns the top **top** display_names;
    concepts whose score is >= score
    """
    return [c['display_name'] for c in concept_list if c['score'] >= score]

In [24]:
dfrecords['keywords'] = dfrecords['content'].apply(get_keywords)
dfrecords['top_concepts'] = dfrecords['concepts'].apply(get_top_concepts)

In [25]:
texts = dfrecords['content'].str.lower().values.tolist()

In [26]:
def get_content_embeddings(dfrecords:pd.DataFrame) -> pd.DataFrame:
    """
    passes the preprocessed mitigation strings
    data through the embedding model to produce the vector
    space representation of each pet mitigation.
    """
    sent = Sentence("The grass is green.")
    DOC_EMBEDDINGS.embed(sent)
    texts = dfrecords["content"].str.lower().values.tolist()
    all_descriptions = np.empty((len(texts), len(sent.embedding)))
    for i in tqdm(range(len(texts))):
        sent = Sentence(texts[i])
        DOC_EMBEDDINGS.embed(sent)
        all_descriptions[i, :] = sent.embedding.cpu().numpy()
        # gc.collect()
        torch.cuda.empty_cache()
    dfcontentvectors = pd.DataFrame.from_records(all_descriptions, index=dfrecords.index)
    return dfcontentvectors

In [27]:
dfcontentvectors = get_content_embeddings(dfrecords)

100%|███████████████████████████████████████████████████████████████████| 22370/22370 [08:01<00:00, 46.42it/s]


In [28]:
#umap.UMAP?
N_COMPONENTS = 2 # can visualize this way
umap_reducer = umap.UMAP(n_components=N_COMPONENTS,
                       #  metric='euclidean')
                         random_state=1234,
                         metric='cosine')  # can experiment with this metric as well as the other 
# parameters
# to see what other literature is in the same information space, we need to keep this umap_reducer 
# object as well as the gmm model below.

# Apply UMAP to the vectorized strings
reduced_vectors = umap_reducer.fit_transform(dfcontentvectors.to_numpy())
dfreduced = pd.DataFrame.from_records(reduced_vectors, 
                index=dfcontentvectors.index)
dfreduced.columns = ['x','y']

/home/davidd/.local/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


## Use hdbscan to cluster

In [29]:
import hdbscan

hdbscan_args = {'min_cluster_size': 15,
                            'metric': 'euclidean',
                            'cluster_selection_method': 'eom',
                            'cluster_selection_epsilon': 0.1
               }

cluster = hdbscan.HDBSCAN(**hdbscan_args).fit(dfreduced[['x','y']].to_numpy())

dfreduced['cluster'] = cluster.labels_
dfreduced['probability'] = cluster.probabilities_

dfpapers = dfrecords.merge(dfreduced, left_index=True,
                           right_index=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [30]:
#help(dfpapers.explode)
del dfpapers['id']
dfstart = dfpapers.reset_index()
dfstart.head()

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,...,created_date,fulltext_origin,is_authors_truncated,abstract,content,top_concepts,x,y,cluster,probability
0,https://openalex.org/W3152893301,https://doi.org/10.1016/j.aiopen.2021.01.001,Graph neural networks: A review of methods and...,Graph neural networks: A review of methods and...,2020,2020-01-01,{'openalex': 'https://openalex.org/W3152893301...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",review,...,2021-04-26,NaN,NaN,Lots of learning tasks require dealing with gr...,Graph neural networks: A review of methods and...,[Computer science],12.073812,5.613014,147,0.730557
1,https://openalex.org/W2911286998,https://doi.org/10.1145/3308558.3313562,Heterogeneous Graph Attention Network,Heterogeneous Graph Attention Network,2019,2019-05-13,{'openalex': 'https://openalex.org/W2911286998...,en,"{'is_oa': False, 'landing_page_url': 'https://...",preprint,...,2019-02-21,ngrams,NaN,"Graph neural network, as a powerful graph repr...",Heterogeneous Graph Attention Network. Graph n...,"[Computer science, Interpretability]",12.327925,5.635758,147,0.930740
2,https://openalex.org/W2963264685,https://doi.org/10.1016/j.future.2017.08.020,A survey on the security of blockchain systems,A survey on the security of blockchain systems,2020,2020-06-01,{'openalex': 'https://openalex.org/W2963264685...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,2019-07-30,ngrams,NaN,"Since its inception, the blockchain technology...",A survey on the security of blockchain systems...,"[Blockchain, Cryptocurrency, Computer science,...",13.655714,12.477859,167,1.000000
3,https://openalex.org/W3110989801,https://doi.org/10.1007/s11432-020-2955-6,Towards 6G wireless communication networks: vi...,Towards 6G wireless communication networks: vi...,2020,2020-11-24,{'openalex': 'https://openalex.org/W3110989801...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",article,...,2020-12-21,NaN,NaN,Abstract The fifth generation (5G) wireless co...,Towards 6G wireless communication networks: vi...,[Computer science],8.426230,12.004443,160,0.613752
4,https://openalex.org/W2799041689,https://doi.org/10.1109/taffc.2020.2981446,Deep Facial Expression Recognition: A Survey,Deep Facial Expression Recognition: A Survey,2022,2022-07-01,{'openalex': 'https://openalex.org/W2799041689...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,2018-05-07,NaN,NaN,With the transition of facial expression recog...,Deep Facial Expression Recognition: A Survey. ...,"[Deep learning, Overfitting, Artificial intell...",10.678908,3.537422,157,0.812517


In [31]:
dfstart['publication_year'].value_counts(dropna=False)

2023    4724
2022    4383
2021    4017
2019    3988
2020    3915
2024    1343
Name: publication_year, dtype: int64

In [32]:
dfstart.shape

(22370, 55)

In [33]:
dfbig = dfstart.explode(column='authorships')
dfbig.shape, dfstart.shape

((115245, 55), (22370, 55))

In [34]:
dfbig.columns

Index(['id', 'doi', 'title', 'display_name', 'publication_year',
       'publication_date', 'ids', 'language', 'primary_location', 'type',
       'type_crossref', 'indexed_in', 'open_access', 'authorships',
       'countries_distinct_count', 'institutions_distinct_count',
       'corresponding_author_ids', 'corresponding_institution_ids', 'apc_list',
       'apc_paid', 'has_fulltext', 'cited_by_count',
       'cited_by_percentile_year', 'biblio', 'is_retracted', 'is_paratext',
       'primary_topic', 'topics', 'keywords', 'concepts', 'mesh',
       'locations_count', 'locations', 'best_oa_location',
       'sustainable_development_goals', 'grants', 'datasets', 'versions',
       'referenced_works_count', 'referenced_works', 'related_works',
       'ngrams_url', 'cited_by_api_url', 'counts_by_year', 'updated_date',
       'created_date', 'fulltext_origin', 'is_authors_truncated', 'abstract',
       'content', 'top_concepts', 'x', 'y', 'cluster', 'probability'],
      dtype='object')

In [35]:
def add_extra_to_authorships(row: pd.DataFrame):
    """
    row[authorships] is a dictionary;
    add in the id key to that dictionary
    whose value is row[id]
    """
    complete_dict = row["authorships"]
   # assert type(complete_dict) == dict
    #print(type(complete_dict))
    if type(complete_dict) == dict:
        complete_dict["id"] = row["id"]
        complete_dict["x"] = row["x"]
        complete_dict["y"] = row["y"]
        complete_dict["cluster"] = row["cluster"]
        complete_dict["cluster_score"] = row["probability"]
        complete_dict["title"] = row["title"]
        complete_dict["abstract"] = row["abstract"]
        complete_dict["doi"] = row["doi"]
        complete_dict["publication_date"] = row["publication_date"]
        complete_dict["publication_year"] = row["publication_year"]
        complete_dict["grants"] = row["grants"]
        complete_dict["locations"] = row["locations"]
        return complete_dict
    else:
        return row["authorships"]

In [36]:
dfbig['big_authorships'] = dfbig.apply(add_extra_to_authorships, axis=1)

In [37]:
#dfbig['authorships'].tolist()
bigvals = dfbig['authorships'].tolist()

In [38]:
dictvals = [c for c in bigvals if type(c) != float]

In [39]:
dictvals[0]['author'].keys()

dict_keys(['id', 'display_name', 'orcid'])

In [40]:
dftriple = pd.json_normalize(dictvals,
                  record_path=['institutions'],
                  meta=['id','raw_affiliation_strings','author_position', 'doi',
                        'title','abstract','publication_date', 'publication_year',
                        'grants','locations',
                        'is_corrresponding','x','y','cluster','cluster_score',
                       ['author','id'], ['author', 'display_name'],
                       ['author','orcid']],
                  errors='ignore',
                  sep='_',
                  meta_prefix='paper_',
                #  record_prefix='author_'
                 )

In [41]:
dftopics = dfcontentvectors.copy()
dftopics['cluster'] = dfpapers['cluster']
dfmeantopics = dftopics.groupby('cluster').mean().copy()
reduced_topics = umap_reducer.transform(dfmeantopics.to_numpy())
df_reduced_topics = pd.DataFrame.from_records(reduced_topics, 
                index=dfmeantopics.index)
df_reduced_topics.columns = ['x','y']
df_reduced_topics['topic'] = df_reduced_topics.index
df_reduced_topics.head()

def get_cluster_concepts(topic_num:int, n:int=20):
    """
    takes an integer topic_num corresponding to a 
    given topic number and
    returns the list of top n occuring concepts
    from the top_concept field
    """
    top_concepts = dfpapers[dfpapers['cluster'] == topic_num]['top_concepts'].tolist()
    flat_concepts = [item for sublist in top_concepts for item in sublist]
    concepts_dict = {c:flat_concepts.count(c) for c in flat_concepts}
    sorted_concepts = sorted(concepts_dict.items(), key=lambda x:x[1], reverse=True)
    return [c[0] for c in sorted_concepts][:n]

def get_yake_cluster_phrases(topic_num:int, n:int=20):
    """
    takes in an integer n corresponding
    to a given topic number and
    returns the list of keyphrases (TopicRank method)
    """
    documents = dfpapers[dfpapers['cluster'] == topic_num]['content'].tolist()
    topic_input = ". ".join(documents)
    #extractor = pke.unsupervised.TextRank()
    kw_extractor = yake.KeywordExtractor(top=n, stopwords=None)
    keywords = kw_extractor.extract_keywords(topic_input)
    #extractor.load_document(input=topic_input,
    #                    language='en',
    #                    normalization=None)

    #extractor.candidate_selection()

    #window = 2
    #use_stems = False
    #extractor.candidate_weighting(window=window,
    #                          use_stems=use_stems)
    #extractor.candidate_weighting()
    #threshold = 0.8
   # keyphrases = extractor.get_n_best(n=20, threshold=threshold)
    #keyphrases = extractor.get_n_best(n=n)
    return [p[0] for p in keywords]

wikiconcepts = df_reduced_topics['topic'].apply(get_cluster_concepts)

wikikeywords = df_reduced_topics['topic'].apply(get_yake_cluster_phrases)

dfpapers['id'] = dfpapers.index
dfinfo = dfpapers[['x','y','id','title','doi','cluster','grants',
                   'locations',
                 'publication_date','keywords','top_concepts']].copy()

centroids = dfinfo.groupby('cluster')[['x','y']].mean().copy()
centroids['concepts'] = wikiconcepts
centroids['cluster'] = centroids.index
centroids['keywords'] = wikikeywords

In [42]:
def wrap_it(x):
    return "<br>".join(textwrap.wrap(x, width=40))
   # return "<br>".join(textwrap.wrap(x.replace(r'\s+', ' '), width=40))

In [43]:
centroids['wrapped_keywords'] = centroids['keywords'].apply(str).apply(wrap_it)
centroids['wrapped_concepts'] = centroids['concepts'].apply(str).apply(wrap_it)

In [44]:
centroids.to_pickle('updatejammingcentroids2d.pkl')

In [45]:
dftriple.to_pickle('updatejammingdftriple2d.pkl')

In [46]:
def get_affils_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by id, ror sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg.groupby(['id','display_name','country_code',
                     'type'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [47]:
dv84, kw84 = get_affils_cluster_sort(dftriple, 1)
print(kw84)
dv84.head(10)

1
['Molecular Communication', 'Molecular Communication Systems', 'Molecular', 'Communication', 'Diffusion-Based Molecular Communication', 'Signal Detection', 'molecular signals', 'Detection', 'Molecular Signal Detection', 'signal', 'molecular communication relay', 'signal detection schemes', 'molecule shift keying', 'Communication Systems', 'Mobile Molecular Communication', 'Diffusion Molecular Communication', 'channel', 'molecular communication environment', 'Performance', 'molecules']


,,,,paper_cluster_score
id,display_name,country_code,type,
https://openalex.org/I139759216,Beijing University of Posts and Telecommunications,CN,education,46.353437
https://openalex.org/I39555362,University of Warwick,GB,education,21.472349
https://openalex.org/I82284825,Cranfield University,GB,education,15.056173
https://openalex.org/I37987034,Guangzhou University,CN,education,14.116289
https://openalex.org/I4800084,Southwest Jiaotong University,CN,education,6.176405
https://openalex.org/I47508984,Imperial College London,GB,education,6.176405
https://openalex.org/I183067930,Shanghai Jiao Tong University,CN,education,5.0
https://openalex.org/I90610280,South China University of Technology,CN,education,4.0
https://openalex.org/I4210128584,The Alan Turing Institute,GB,facility,3.0


In [48]:
dv84, kw84 = get_affils_cluster_sort(dftriple, 0)
print(kw84)
dv84.head(10)

0
['Coq', 'theorem', 'Coq proof assistant', 'set theory', 'formal proof', 'axiomatic set theory', 'Coq proof', 'Spaces', 'Topological Spaces', 'proof assistant Coq', 'proof', 'space', 'Formal', 'theory', 'set', 'completeness theorem', 'formalization', 'theorems', 'Formal System', 'system']


,,,,paper_cluster_score
id,display_name,country_code,type,
https://openalex.org/I139759216,Beijing University of Posts and Telecommunications,CN,education,37.388068
https://openalex.org/I3130859241,Yili Normal University,CN,education,4.0
https://openalex.org/I172675005,Nanyang Technological University,SG,education,3.053296
https://openalex.org/I4210141689,Yancheng Teachers University,CN,education,2.6702
https://openalex.org/I163151501,Hangzhou Normal University,CN,education,1.0
https://openalex.org/I118574674,Jiangsu Normal University,CN,education,0.895597
https://openalex.org/I9356336,Zhangzhou Normal University,CN,education,0.801589
https://openalex.org/I125839683,Beijing Institute of Technology,CN,education,0.437698


In [49]:
dv84, kw84 = get_affils_cluster_sort(dftriple, 0)
print(kw84)
dv84.head(20)

0
['Coq', 'theorem', 'Coq proof assistant', 'set theory', 'formal proof', 'axiomatic set theory', 'Coq proof', 'Spaces', 'Topological Spaces', 'proof assistant Coq', 'proof', 'space', 'Formal', 'theory', 'set', 'completeness theorem', 'formalization', 'theorems', 'Formal System', 'system']


,,,,paper_cluster_score
id,display_name,country_code,type,
https://openalex.org/I139759216,Beijing University of Posts and Telecommunications,CN,education,37.388068
https://openalex.org/I3130859241,Yili Normal University,CN,education,4.0
https://openalex.org/I172675005,Nanyang Technological University,SG,education,3.053296
https://openalex.org/I4210141689,Yancheng Teachers University,CN,education,2.6702
https://openalex.org/I163151501,Hangzhou Normal University,CN,education,1.0
https://openalex.org/I118574674,Jiangsu Normal University,CN,education,0.895597
https://openalex.org/I9356336,Zhangzhou Normal University,CN,education,0.801589
https://openalex.org/I125839683,Beijing Institute of Technology,CN,education,0.437698


In [50]:
dfinfo = dfpapers[['x','y','id','title','doi','cluster','probability',
                 'publication_date','grants','locations',
                   'keywords','top_concepts']].copy()

In [51]:
# Group by 'paper_id' and concatenate 'paper_raw_affiliation_strings'
grouped = dftriple.groupby('paper_id')['paper_raw_affiliation_strings'].apply(lambda x: list(set([item for sublist in x for item in sublist]))).reset_index()

# Convert the series back to a dictionary
pap_affils_dict = grouped.set_index('paper_id')['paper_raw_affiliation_strings'].to_dict()

In [52]:
import itertools


In [53]:
pap_authors_dict = dftriple.groupby('paper_id')['paper_author_display_name'].apply(lambda x: x.values)

In [54]:
dfinfo['affil_list'] = dfinfo['id'].map(pap_affils_dict)

In [55]:
dfinfo['author_list'] = dfinfo['id'].map(pap_authors_dict)

In [56]:
dfinfo['wrapped_affil_list'] = dfinfo['affil_list'].apply(str).apply(wrap_it)
dfinfo['wrapped_author_list'] = dfinfo['author_list'].apply(str).apply(wrap_it)

In [57]:
dfinfo['wrapped_keywords'] = dfinfo['keywords'].apply(str).apply(wrap_it)

In [58]:
def get_source_name(loc_list):
    """
    grab the first item in the list;
    retturn the display name
    """
    try:
        primary = loc_list[0]
        return primary["source"]["display_name"]
    except:
        return None

def get_source_type(loc_list):
    """
    grab the first item in the list;
    return the source type
    """
    try:
        primary = loc_list[0]
        return primary["source"]["type"]
    except:
        return None

In [59]:
dfinfo["source"] = dfinfo["locations"].apply(get_source_name)
dfinfo["source_type"] = dfinfo["locations"].apply(get_source_type)

In [60]:
dfinfo.to_pickle('updatejammingdfinfo2d.pkl')

In [61]:
def get_funder_names(funder_list):
    """
    funder_list is a list of dictionaries
    with three keys; return the list of 
    unique **funder_display_name**
    values
    """
    try:
        funder_names = list(set([f['funder_display_name'] for f in funder_list]))
        return funder_names    
    except:
        return []

In [62]:
dftriple["source"] = dftriple["paper_locations"].apply(get_source_name)
dftriple["source_type"] = dftriple["paper_locations"].apply(get_source_type)
dftriple["funder_list"] = dftriple["paper_grants"].apply(get_funder_names)

In [63]:
dftriple.to_pickle('updatejammingdftriple2d.pkl')

In [64]:
def get_journals_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by source (where
    source_type == 'journal') sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg[dg['source_type'] == 'journal'].groupby(['source'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [65]:
def get_conferences_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by source (where
    source_type == 'journal') sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg[dg['source_type'] == 'conference'].groupby(['source'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [66]:
def get_country_collaborations_sort(dc:pd.DataFrame, cl:int):
    """
    resticts the dataframe dc to cluster value cl
    and returns the results of paper_id s where there is 
    more than one country_code
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    dv = dg.groupby('paper_id')['country_code'].apply(lambda x: len(set(x.values))).to_frame()
    dc = dg.groupby('paper_id')['country_code'].apply(lambda x: list(set(x.values))).to_frame()
    dc.columns = ['collab_countries']
    dv.columns = ['country_count']
    dv['collab_countries'] = dc['collab_countries']
    dv.sort_values('country_count',ascending=False, inplace=True)
    di = dfinfo.loc[dv.index].copy()
    di['country_count'] = dv['country_count']
    di['collab_countries'] = dv['collab_countries']
    return di[di['country_count'] > 1]

In [67]:
dv = get_country_collaborations_sort(dftriple, 0)
dv

,x,y,id,title,doi,cluster,probability,publication_date,grants,locations,...,top_concepts,affil_list,author_list,wrapped_affil_list,wrapped_author_list,wrapped_keywords,source,source_type,country_count,collab_countries
paper_id,,,,,,,,,,,,,,,,,,,,,
https://openalex.org/W4206819778,1.105263,3.708296,https://openalex.org/W4206819778,Maximal point spaces of posets with relative l...,https://doi.org/10.2298/fil2108645s,0,0.895597,2021-01-01,[],"[{'is_oa': True, 'landing_page_url': 'https://...",...,"[Partially ordered set, Mathematics, Bounded f...","[Mathematics and Mathematical Education, Natio...","[Chong Shen, Xiaoyong Xi, Dongsheng Zhao]","['Mathematics and Mathematical<br>Education, N...",['Chong Shen' 'Xiaoyong Xi' 'Dongsheng<br>Zhao'],"['lower topology', 'poset', 'space',<br>'relat...",Filomat,journal,2,"[CN, SG]"
https://openalex.org/W4328130573,1.108287,3.715919,https://openalex.org/W4328130573,Further studies on open well-filtered spaces,https://doi.org/10.46298/entics.10392,0,0.843608,2023-03-21,[],"[{'is_oa': True, 'landing_page_url': 'https://...",...,"[Open set, Mathematics, Space (punctuation)]","[Mathematics and Mathematics Education, Nation...","[Chong Shen, Xiaoyong Xi, Dongsheng Zhao]","['Mathematics and Mathematics Education,<br>Na...",['Chong Shen' 'Xiaoyong Xi' 'Dongsheng<br>Zhao'],"['open well-filtered spaces', 'open<br>well-fi...",Electronic notes in theorical informatics and ...,journal,2,"[CN, SG]"
https://openalex.org/W4327558482,1.081017,3.732934,https://openalex.org/W4327558482,On a problem about strictly completely regular...,https://doi.org/10.1016/j.topol.2023.108489,0,0.512501,2023-05-01,[],"[{'is_oa': False, 'landing_page_url': 'https:/...",...,"[Mathematics, Counterexample, Partially ordere...","[School of Mathematics and Statistics, Yanchen...","[Xiaoyong Xi, Chong Shen, Gaolin Li, Dongsheng...","['School of Mathematics and Statistics,<br>Yan...",['Xiaoyong Xi' 'Chong Shen' 'Gaolin Li'<br>'Do...,"['completely regular ordered', 'strictly<br>co...",Topology and its applications,journal,2,"[CN, SG]"
https://openalex.org/W4283586870,1.102607,3.713924,https://openalex.org/W4283586870,Hofmann-Mislove type definitions of non-Hausdo...,https://doi.org/10.1017/s0960129522000196,0,0.801589,2022-01-01,[],"[{'is_oa': False, 'landing_page_url': 'https:/...",...,"[Hausdorff space, Mathematics, Pure mathematic...","[School of Mathematics and Statistics, Minnan ...","[Chong Shen, Xiaoyong Xi, Xiaoquan Xu, Dongshe...","['School of Mathematics and Statistics,<br>Min...",['Chong Shen' 'Xiaoyong Xi' 'Xiaoquan<br>Xu' '...,"['Psi', 'Hofmann-Mislove type<br>definitions',...",MSCS. Mathematical structures in computer scie...,journal,2,"[CN, SG]"


In [68]:
import networkx as nx
from pyvis.network import Network
import igraph as ig # for getting a layout w/o relying on slow pyvis physics 

In [69]:
dfinfo["funder_list"] = dfinfo["grants"].apply(get_funder_names)
dfinfo["wrapped_funder_list"] = dfinfo["funder_list"].apply(str).apply(wrap_it)

In [70]:
dfinfo.to_pickle('updatejammingdfinfo2d.pkl')

In [71]:
dfinfo[['id','keywords','wrapped_keywords','wrapped_funder_list']].head()

,id,keywords,wrapped_keywords,wrapped_funder_list
id,,,,
https://openalex.org/W3152893301,https://openalex.org/W3152893301,"[review of methods, Graph neural networks, Gra...","['review of methods', 'Graph neural<br>network...",['National Key Research and Development<br>Pro...
https://openalex.org/W2911286998,https://openalex.org/W2911286998,"[Graph neural network, neural network, Graph n...","['Graph neural network', 'neural<br>network', ...",[]
https://openalex.org/W2963264685,https://openalex.org/W2963264685,"[blockchain systems, blockchain, systems, secu...","['blockchain systems', 'blockchain',<br>'syste...","['China Postdoctoral Science<br>Foundation', '..."
https://openalex.org/W3110989801,https://openalex.org/W3110989801,"[wireless communication networks, communicatio...","['wireless communication networks',<br>'commun...",[]
https://openalex.org/W2799041689,https://openalex.org/W2799041689,"[Facial Expression Recognition, Deep Facial Ex...","['Facial Expression Recognition', 'Deep<br>Fac...",['National Natural Science Foundation of<br>Ch...


In [72]:
kw_dict = dfinfo['keywords'].to_dict()

In [73]:
dc = dftriple[dftriple['paper_cluster'] == 10].copy()
dc.shape

(445, 27)

In [74]:
[x for row in dc['funder_list'].tolist() for x in row]

['Fundamental Research Funds for the Central Universities',
 'Aeronautical Science Foundation of China',
 'Fundamental Research Funds for the Central Universities',
 'Aeronautical Science Foundation of China',
 'Fundamental Research Funds for the Central Universities',
 'Aeronautical Science Foundation of China',
 'Fundamental Research Funds for the Central Universities',
 'Aeronautical Science Foundation of China',
 'Fundamental Research Funds for the Central Universities',
 'Aeronautical Science Foundation of China',
 'Fundamental Research Funds for the Central Universities',
 'Aeronautical Science Foundation of China',
 'Fundamental Research Funds for the Central Universities',
 'Aeronautical Science Foundation of China',
 'Fundamental Research Funds for the Central Universities',
 'Aeronautical Science Foundation of China',
 'Fundamental Research Funds for the Central Universities',
 'Aeronautical Science Foundation of China',
 'Special Project for Research and Development in Key a

In [75]:
kw_dict = dfinfo['keywords'].to_dict()

# add in the affiliations as nodes as well; that row, author, paper, affil. all three get links. ok.
def create_nx_graph(df: pd.DataFrame, cl:int) -> nx.Graph:
    """
    takes the dataframe df, and creates the undirected graph
    from the source and target columns for each row.
    """
    g = nx.Graph() # dc['paper_cluster'] == cl
    dc = df[df['paper_cluster'] == cl]
    author_counts = dc['paper_author_id'].tolist()
    author_counts_dict = {c:author_counts.count(c) for c in author_counts}
    affiliation_counts = dc['id'].tolist()
    affiliation_counts_dict = {c:affiliation_counts.count(c) for c in affiliation_counts}
    source_counts = dc['source'].tolist()
    source_counts_dict = {c:source_counts.count(c) for c in source_counts}
    funder_counts = [x for row in dc['funder_list'].tolist() for x in row]
    funder_counts_dict = {c:funder_counts.count(c) for c in funder_counts}
    for index, row in df[df['paper_cluster'] == cl].iterrows():
        g.add_node(row['paper_id'], group='work', title=row['paper_title'])
        g.add_node(row['paper_author_id'], title=row['paper_author_display_name'],
                   group='author',value = author_counts_dict[row['paper_author_id']])
        g.add_node(row['id'], group='affiliation',
                   title=row['display_name'] + '\n' + row['country_code'],
                  value = affiliation_counts_dict[row['id']])
        if row['source']:
            g.add_node(row['source'], group=row['source_type'],
                      title=row['source'] + ' :\n ' + row['source_type'],
                      value=source_counts_dict[row['source']])
            g.add_edge(
                row['paper_id'],
                row['source'],
                title=row['paper_title'] + ' :\n ' + str(row['paper_publication_date']) +  \
                ' :\n' + row['source'] + ' :\n ' + \
                row['source_type'],
              #  weight = df[(df['paper_id'] == row['paper_id']) & \
              #              (df['source'] == row['source'])]['paper_cluster_score'].sum()
               # weight = row['paper_cluster_score']
            )
            g.add_edge(
                row['paper_author_id'],
                row['source'],
                title=row['paper_author_display_name'] + ':\n' + row['source'],
             #   weight = df[(df['paper_author_id'] == row['paper_author_id']) & \
              #              (df['source'] == row['source'])]['paper_cluster_score'].sum()
               # weight = row['paper_cluster_score']
            )
        if len(row['funder_list']) > 0:
            for f in row['funder_list']:
                g.add_node(f, group='funder',
                          title=str(f),
                          value = founder_counts_dict[f]),
                g.add_edge(
                       row['paper_id'],
                       f,
                       title=row['paper_title'] + ':\n ' +  str(row['paper_publication_date']) + \
                       ' :\n' + str(f),
                  #  weight = row['paper_cluster_score']
                   )
                g.add_edge(
                       f,
                       row['paper_author_id'],
                       title=row['paper_author_display_name'] + ' :\n ' + \
                       str(f),
                  #  weight = row['paper_cluster_score']
                       
                   )
                g.add_edge(
                       f,
                       row['id'],
                       title=row['display_name'] + '\n' + row['country_code'] + ' :\n ' + \
                       str(f)  ,
                  #  weight = row['paper_cluster_score']
                   )  
                if row["source"]:
                    g.add_edge(
                        f,
                        row["source"],
                        title=row["source"] + ' :\n' + str(f),
                     #   weight = row['paper_cluster_score']
                    )
        g.nodes[row['paper_id']]['title'] = (
            row['paper_title'] + ' :\n ' + str(row['paper_publication_date'] + ':\n' + 
            '\n'.join(kw_dict[row['paper_id']]))
        )
        g.nodes[row['paper_author_id']]['title'] = (
            row['paper_author_display_name']
        )
        g.add_edge(
            row['paper_id'],
            row['paper_author_id'],
        title=row['paper_title'] + ' :\n ' + row['paper_author_display_name'] + ' :\n ' + \
            row['paper_raw_affiliation_string'],
         #   weight = row['paper_cluster_score']
        )
        g.add_edge(
            row['paper_author_id'],
            row['id'],
            title=row['paper_author_display_name'] + ' :\n ' + \
            row['display_name'] + ' :\n ' + row['country_code'],
          #  weight = row['paper_cluster_score']
        )
        g.add_edge(
            row['paper_id'],
            row['id'],
            title=row['paper_title'] + ' :\n ' + str(row['paper_publication_date']) + ':\n' + 
            row['display_name'] + ' :\n ' + row['country_code'],
         #   weight = row['paper_cluster_score']
        )
        
    g_ig = ig.Graph.from_networkx(g) # assign 'x', and 'y' to g before returning
    #layout = g_ig.layout_auto()
    #layout = g_ig.layout_davidson_harel()
    layout = g_ig.layout_umap(min_dist = 2, epochs = 500)
    # https://igraph.org/python/tutorial/0.9.6/visualisation.html
    coords = layout.coords
    allnodes = list(g.nodes())
    coords_dict = {allnodes[i]:(coords[i][0], coords[i][1]) for i in range(len(allnodes))}
    for i in g.nodes():
        g.nodes[i]['x'] = 250 * coords_dict[i][0] # the scale factor needed 
        g.nodes[i]['y'] = 250 * coords_dict[i][1]
    return g

In [76]:
def create_pyvis_html(cl: int, filename: str = "pyvis_coauthorships_graph.html"):
    """
    wrapper function that calls create_nx_graph to finally 
    produce an interactive pyvis standalone html file
    """
    g_nx = create_nx_graph(dftriple, cl);
    h = Network(height="1000px",
          #  heading="Mitigations and Techniques Relationships",
                width="100%",
                cdn_resources="remote", # can grab the visjs library to make this local if needed
            # probably should
                bgcolor="#222222",
            neighborhood_highlight=True,
              # default_node_size=1,
                font_color="white",
                directed=False,
               # select_menu=True,
                filter_menu=True,
                notebook=False,
               )
    #h.repulsion()
    h.from_nx(g_nx, show_edge_weights=False)
    #h.barnes_hut()
    #h.repulsion(node_distance=40,
    #            central_gravity=-0.2, spring_length=5, spring_strength=0.005, damping=0.09)
    neighbor_map = h.get_adj_list()
   # for node in h.nodes:
   #     if node['group'] == 'author':
   #         a = list(neighbor_map[node["id"]]) # want to insert a "\n" into every third element of a
   #     if node['group'] == 'work':
   #         a = list(neighbor_map[node["id"]])
   #     i = 3
   #     while i < len(a):
   #         a.insert(i, "\n")
   #         i += 4
   #     node["title"] += "\n Neighbors: \n" + " | ".join(a)
   #     node["value"] = len(neighbor_map[node["id"]]) 
# "physics": {
#    "enabled": false
#  },
    h.set_options(
    """
const options = {
  "interaction": {
    "navigationButtons": false
  },
 "physics": {
     "enabled": false
 },
  "edges": {
    "color": {
        "inherit": true
    },
    "setReferenceSize": null,
    "setReference": {
        "angle": 0.7853981633974483
    },
    "smooth": {
        "forceDirection": "none"
    }
  }
  }
    """
    )
    #h.show_buttons(filter_=['physics'])
  #  h.barnes_hut()
    #h.repulsion()
    try:
        path = './tmp'
        h.save_graph(f"{path}/{filename}")
        HtmlFile = open(f"{path}/{filename}","r",
                        encoding='utf-8')
    except:
        h.save_graph(f"{filename}")
        HtmlFile = open(f"{filename}", "r",
                        encoding="utf-8")
    return h

In [78]:
#del dfinfo['authors_list']

In [79]:
dfinfo.shape

(22370, 21)

In [80]:
dftime = dfinfo[['cluster','probability','publication_date']].copy()

In [81]:
dftime['publication_datetime'] = pd.to_datetime(dftime['publication_date'])

In [82]:
def get_time_series(dg, cl:int):
    """
    takes dg and the cluster number cl
    and returns a time series chart
    by month, y-axis is the article count
    """
    dftime = dg[dg.cluster == cl][['cluster','probability','publication_date']].copy()
    dftime['date'] = pd.to_datetime(dftime['publication_date'])
    dftime.sort_values('date', inplace=True)
    #by_month = pd.to_datetime(dftime['date']).dt.to_period('M').value_counts().sort_index()
    #by_month.index = pd.PeriodIndex(by_month.index)
    #df_month = by_month.rename_axis('month').reset_index(name='counts')
    return dftime

In [83]:
import altair as alt
#alt.data_transformers.enable("data_server")

In [84]:
sources_list = dftriple['source'].unique().tolist()
type(sources_list), len(sources_list)

(list, 1868)

In [85]:
def get_source_json(s:str):
    """
    s is an openalex Sources display_name
    return that Sources object
    """
    source_json = Sources().search_filter(display_name = s).get()
    a = source_json[0]['type']
    if "homepage_url" in source_json[0] and source_json[0]['homepage_url']:
        print(f"{s} has homepage_url and type {source_json[0]['type']}")
        return source_json[0]["homepage_url"]
    else:
        return None

In [86]:
sources_list[5]

'IEEE transactions on knowledge and data engineering'

In [87]:
sj0 = get_source_json(sources_list[5])
sj0

IEEE transactions on knowledge and data engineering has homepage_url and type journal


'https://www.computer.org/web/tkde'

In [88]:
def get_display_page_dict(sl:list):
    """
    sl is a list of Sources display_name values
    returns the dictionary mapping
    display_names with homepage_url values."""
    mapping_dict = dict()
    for s in tqdm(sl):
        try:
            mapping_dict[s] = get_source_json(s)
        except:
            pass
    return mapping_dict

In [89]:
source_page_dict = get_display_page_dict(sources_list)

  0%|                                                                        | 1/1868 [00:00<17:59,  1.73it/s]

AI open has homepage_url and type journal


  0%|                                                                        | 3/1868 [00:01<15:30,  2.00it/s]

Future generation computer systems has homepage_url and type journal


  0%|▏                                                                       | 4/1868 [00:02<16:11,  1.92it/s]

...

100%|████████████████████████████████████████████████████████████████████▉| 1866/1868 [19:25<00:01,  1.76it/s]

International journal of system control and information processing has homepage_url and type journal


100%|████████████████████████████████████████████████████████████████████▉| 1867/1868 [19:27<00:00,  1.37it/s]

Imaging science journal/The imaging science journal has homepage_url and type journal


100%|█████████████████████████████████████████████████████████████████████| 1868/1868 [19:27<00:00,  1.60it/s]


In [90]:
len(source_page_dict)

1597

In [91]:
import pickle

with open("updatesource_page_dict.pkl", "wb") as f:
    pickle.dump(source_page_dict, f)

In [92]:
with open("updatesource_page_dict.pkl", "rb") as f:
    source_dict = pickle.load(f)

In [93]:
def get_journals_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by source (where
    source_type == 'journal') sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg[dg['source_type'] == 'journal'].groupby(['source'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    dv['journal'] = dv.index
    dv['hompage_url'] = dv['journal'].map(source_dict)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [94]:
dv, kw = get_journals_cluster_sort(dftriple, 4)
dv.head()

4


,paper_cluster_score,journal,hompage_url
source,,,
IEEE transactions on information theory,19.0,IEEE transactions on information theory,http://www.comm.utoronto.ca/trans-it/
"IEICE transactions on fundamentals of electronics, communications and computer sciences/IEICE transactions on fundamentals of electronics, communications and computer science",9.0,IEICE transactions on fundamentals of electron...,NaN
IEEE access,8.0,IEEE access,http://ieeexplore.ieee.org/servlet/opac?punumb...
Applied mathematics and computation,6.0,Applied mathematics and computation,https://www.journals.elsevier.com/applied-math...
Advances in mathematics of communications,3.0,Advances in mathematics of communications,http://www.aimsciences.org/journals/amc


In [95]:
def get_conferences_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by source (where
    source_type == 'journal') sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg[dg['source_type'] == 'conference'].groupby(['source'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    dv['conference'] = dv.index
    dv['homepage_url'] = dv['conference'].map(source_dict)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [96]:
affils_list = dftriple['display_name'].unique().tolist()
type(affils_list), len(affils_list)

(list, 2649)

In [97]:
 affil_json = Institutions().search_filter(display_name = affils_list[0]).get()

In [98]:
affils_list = dftriple['display_name'].unique().tolist()
type(affils_list), len(affils_list)

affil_json = Institutions().search_filter(display_name = affils_list[1]).get()

def get_affil_json(s:str):
    """
    s is an openalex Sources display_name
    return that Sources object
    """
    affil_json = Institutions().search_filter(display_name = s).get()
    #a = source_json[0]['type']
    if "geo" in affil_json[0]:
       # print(f"{s} has homepage_url and type {source_json[0]['type']}")
        return affil_json[0]["geo"]["latitude"], affil_json[0]["geo"]["longitude"]
    else:
        return None, None
    
def get_display_geo_dict(sl:list):
    """
    sl is a list of Institution display_name values
    returns the dictionary mapping
    display_names with (latitude, longitude) values."""
    mapping_dict = dict()
    for s in tqdm(sl):
        try:
            mapping_dict[s] = get_affil_json(s)
        except:
            pass
    return mapping_dict

affil_geo_dict = get_display_geo_dict(affils_list)


import pickle

with open("updateaffil_geo_dict.pkl", "wb") as f:
    pickle.dump(affil_geo_dict, f)

100%|█████████████████████████████████████████████████████████████████████| 2649/2649 [25:07<00:00,  1.76it/s]


In [99]:
len(affil_geo_dict)

2567

In [100]:
dfinfo['abstract'] = dfinfo['title'].map(dftriple.set_index('paper_title')['paper_abstract'].to_dict())

In [101]:
dfinfo.to_pickle('updatejammingdfinfo2d.pkl')